**PySpark Assignment – Product Sales Analysis**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ProductSalesAnalysis").getOrCreate()

**Part 2: Load Sales Data from CSV**

Read the file into a PySpark DataFrame with header and inferred schema.

Print schema and show top 5 rows.

In [2]:
data = """OrderID,Product,Category,Quantity,UnitPrice,Region
1001,Mobile,Electronics,2,15000,North
1002,Laptop,Electronics,1,55000,South
1003,T-Shirt,Apparel,3,500,East
1004,Jeans,Apparel,2,1200,North
1005,TV,Electronics,1,40000,West
1006,Shoes,Footwear,4,2000,South
1007,Watch,Accessories,2,3000,East
1008,Headphones,Electronics,3,2500,North
"""

with open("sales.csv", "w") as f:
    f.write(data)

df = spark.read.csv("sales.csv", header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- OrderID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: integer (nullable = true)
 |-- Region: string (nullable = true)

+-------+-------+-----------+--------+---------+------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|
+-------+-------+-----------+--------+---------+------+
|   1001| Mobile|Electronics|       2|    15000| North|
|   1002| Laptop|Electronics|       1|    55000| South|
|   1003|T-Shirt|    Apparel|       3|      500|  East|
|   1004|  Jeans|    Apparel|       2|     1200| North|
|   1005|     TV|Electronics|       1|    40000|  West|
+-------+-------+-----------+--------+---------+------+
only showing top 5 rows



 **Part 3: Business Questions**
1. Add a new column TotalPrice = Quantity × UnitPrice

In [3]:
from pyspark.sql.functions import col

df = df.withColumn("TotalPrice", col("Quantity") * col("UnitPrice"))
df.show()

+-------+----------+-----------+--------+---------+------+----------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+----------+-----------+--------+---------+------+----------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|
+-------+----------+-----------+--------+---------+------+----------+



2. Total revenue generated across all regions.

In [4]:
df.agg({"TotalPrice": "sum"}).show()

+---------------+
|sum(TotalPrice)|
+---------------+
|         150400|
+---------------+



3. Category-wise revenue sorted in descending order.

In [5]:
df.groupBy("Category").sum("TotalPrice").orderBy("sum(TotalPrice)", ascending=False).show()

+-----------+---------------+
|   Category|sum(TotalPrice)|
+-----------+---------------+
|Electronics|         132500|
|   Footwear|           8000|
|Accessories|           6000|
|    Apparel|           3900|
+-----------+---------------+



4. Region with the highest number of orders

In [6]:
from pyspark.sql.functions import count

df.groupBy("Region").agg(count("OrderID").alias("OrderCount")).orderBy("OrderCount", ascending=False).show(1)

+------+----------+
|Region|OrderCount|
+------+----------+
| North|         3|
+------+----------+
only showing top 1 row



5. Average Unit Price per Category

In [7]:
df.groupBy("Category").avg("UnitPrice").show()

+-----------+--------------+
|   Category|avg(UnitPrice)|
+-----------+--------------+
|    Apparel|         850.0|
|Electronics|       28125.0|
|   Footwear|        2000.0|
|Accessories|        3000.0|
+-----------+--------------+



6. All orders where TotalPrice is more than 30,000

In [8]:
df.filter(col("TotalPrice") > 30000).show()

+-------+-------+-----------+--------+---------+------+----------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|
+-------+-------+-----------+--------+---------+------+----------+
|   1002| Laptop|Electronics|       1|    55000| South|     55000|
|   1005|     TV|Electronics|       1|    40000|  West|     40000|
+-------+-------+-----------+--------+---------+------+----------+



**Part 4: Data Transformations**
1. Create a new column HighValueOrder which is "Yes" if TotalPrice > 20,000,
else "No" .

In [9]:
from pyspark.sql.functions import when

df = df.withColumn("HighValueOrder", when(col("TotalPrice") > 20000, "Yes").otherwise("No"))
df.show()

+-------+----------+-----------+--------+---------+------+----------+--------------+
|OrderID|   Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+----------+-----------+--------+---------+------+----------+--------------+
|   1001|    Mobile|Electronics|       2|    15000| North|     30000|           Yes|
|   1002|    Laptop|Electronics|       1|    55000| South|     55000|           Yes|
|   1003|   T-Shirt|    Apparel|       3|      500|  East|      1500|            No|
|   1004|     Jeans|    Apparel|       2|     1200| North|      2400|            No|
|   1005|        TV|Electronics|       1|    40000|  West|     40000|           Yes|
|   1006|     Shoes|   Footwear|       4|     2000| South|      8000|            No|
|   1007|     Watch|Accessories|       2|     3000|  East|      6000|            No|
|   1008|Headphones|Electronics|       3|     2500| North|      7500|            No|
+-------+----------+-----------+--------+---------+------+-------

2. Filter and display all high-value orders in the North region.

In [10]:
df.filter((col("HighValueOrder") == "Yes") & (col("Region") == "North")).show()

+-------+-------+-----------+--------+---------+------+----------+--------------+
|OrderID|Product|   Category|Quantity|UnitPrice|Region|TotalPrice|HighValueOrder|
+-------+-------+-----------+--------+---------+------+----------+--------------+
|   1001| Mobile|Electronics|       2|    15000| North|     30000|           Yes|
+-------+-------+-----------+--------+---------+------+----------+--------------+



3. Count how many high-value orders exist per region.

In [11]:
df.filter(col("HighValueOrder") == "Yes").groupBy("Region").count().show()

+------+-----+
|Region|count|
+------+-----+
| South|    1|
|  West|    1|
| North|    1|
+------+-----+



**Part 5: Save Results**

Save the transformed DataFrame as a CSV file named high_value_orders.csv with
headers.

In [12]:
df.filter(col("HighValueOrder") == "Yes") \
  .write.option("header", True) \
  .csv("/content/high_value_orders.csv")